In [ ]:
import sys
import os

# Get the current working directory and navigate to project root
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# From backend folder, go up one level to reach project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))
print(f"Project root: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# Verify the utils directory exists
utils_path = os.path.join(project_root, 'utils')
print(f"Utils path exists: {os.path.exists(utils_path)}")

try:
    from utils import save_artifact, load_artifact, clean_llm_output
    from langchain_openai import ChatOpenAI
    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain
    print("Successfully imported utils functions!")
except ImportError as e:
    print(f"Import error: {e}")
    print(f"sys.path: {sys.path}")

# Try to load API endpoint from different possible locations
app_code = None
possible_locations = [
    "app/main.py",
    "artifacts/app/main.py", 
    "artifacts/convisoft_API_in_memory.py"
]

for location in possible_locations:
    try:
        app_code = load_artifact(location)
        if app_code:
            print(f"Successfully loaded app code from: {location}")
            break
    except:
        continue

if not app_code:
    print("Error: Could not find any FastAPI application code.")
    print("Please make sure you have generated the API code first.")
    raise FileNotFoundError("No FastAPI application code found")

llm = ChatOpenAI(model="gpt-4o")

# Step 1: Generate happy path tests
print("=== Generating Happy Path Tests ===")
happy_path_prompt = PromptTemplate.from_template(
    """Using the provided app code, act as a QA engineer and generate comprehensive happy path tests for the FastAPI application.
The tests should cover the following:
- All CRUD operations for each endpoint.
- Validation of request and response data using Pydantic models.
- Check for status codes being returned such as 201 Created, 200 OK, etc.
Use pytest and FastAPI's TestClient for the tests.

App Code:
{app_code}"""
)

happy_path_chain = LLMChain(llm=llm, prompt=happy_path_prompt)
happy_path_result = happy_path_chain.invoke(app_code=app_code)

print("\n--- Happy Path Tests ---\n")
print(happy_path_result)

# Step 2: Generate edge case tests
print("\n=== Generating Edge Case Tests ===")
edge_case_prompt = PromptTemplate.from_template(
    """Now that the happy path tests have been generated, continue acting as a QA engineer and generate comprehensive edge case tests for the FastAPI application.
    The tests should cover the following:
    - Invalid data inputs (e.g., missing required fields, invalid formats).
    - Non-existent resources (e.g., fetching or deleting an item that doesn't exist).
    - Boundary conditions (e.g., maximum field lengths, zero or negative values).
    - 404 Not Found, 400 Bad Request, and other error status codes.
Use pytest and FastAPI's TestClient for the tests.
Combine these edge cases with the previously generated happy path tests.

Happy Path Tests:
{happy_path_tests}

App Code:
{app_code}"""
)

edge_case_chain = LLMChain(llm=llm, prompt=edge_case_prompt)
edge_case_result = edge_case_chain.invoke(happy_path_tests=happy_path_result, app_code=app_code)

print("\n--- Edge Case Tests ---\n")
print(edge_case_result)

# Step 3: Generate database fixture
print("\n=== Generating Database Fixture ===")
db_fixture_prompt = PromptTemplate.from_template(
    """Using the provided app code, generate a pytest fixture that sets up an isolated test database for the FastAPI application.
The fixture should:
- Create a temporary SQLite database in memory (:memory:)
- Override the get_db dependency in the app to use this test database session
- Create all the database tables before the tests run
- Drop all the database tables after the tests complete
- Provide a database session to the tests that is rolled back after each test to ensure isolation

App Code:
{app_code}"""
)

db_fixture_chain = LLMChain(llm=llm, prompt=db_fixture_prompt)
db_fixture_result = db_fixture_chain.invoke(app_code=app_code)

print("\n--- DB Fixture ---\n")
print(db_fixture_result)

# Step 4: Refactor tests to use the DB fixture
print("\n=== Refactoring Tests with Database Fixture ===")
refactor_tests_prompt = PromptTemplate.from_template(
    """Using the provided app code, unit tests, and the generated pytest fixture, refactor the tests to utilize the test database fixture.
Ensure that the tests now use the fixture for database interactions instead of the original get_db dependency.
Combine everything into a complete test file that includes:
1. The database fixture
2. All the happy path tests
3. All the edge case tests
4. Proper imports and setup

Only return the final complete test file content.
Do not include any explanations or extra text.

App Code:
{app_code}

Unit Tests:
{unit_tests}

Database Fixture:
{db_fixture}"""
)

refactor_tests_chain = LLMChain(llm=llm, prompt=refactor_tests_prompt)
refactored_tests_result = refactor_tests_chain.invoke(
    app_code=app_code,
    unit_tests=edge_case_result,
    db_fixture=db_fixture_result
)

print("\n--- Refactored Tests ---\n")
print(refactored_tests_result)

# Clean and save the final tests
cleaned_code = clean_llm_output(refactored_tests_result, language='python')
save_artifact(cleaned_code, "app/test_main.py", overwrite=True)
print("\n✅ Tests saved to app/test_main.py")

Current working directory: c:\Users\labadmin\Desktop\Labs\KMSH_contracting\backend
Project root: c:\Users\labadmin\Desktop\Labs\KMSH_contracting
Utils path exists: True
Successfully imported utils functions!
Successfully loaded app code from: app/main.py
=== Generating Happy Path Tests ===


C:\Users\labadmin\AppData\Local\Temp\2\ipykernel_2976\3921252164.py:69: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  happy_path_result = happy_path_chain.run(app_code=app_code)



--- Happy Path Tests ---

To test the FastAPI application, you'll want to implement tests for the described CRUD operations and ensure validation with Pydantic models and expected status codes. Below is an outline of how you might structure such tests using `pytest` and FastAPI's `TestClient`. This assumes you have the app running in a test environment where the database is either mocked or reset after each test:

```python
# test_main.py
import pytest
from fastapi.testclient import TestClient
from main import app

client = TestClient(app)

# Dummy data for testing
new_user_data = {
    "username": "newuser",
    "email": "newuser@example.com",
    "password": "newpassword"
}

new_company_data = {
    "legal_name": "Test Company"
}

new_contract_data = {
    "contract_number": "CN123456",
    "title": "Test Contract",
    "company_id": 1,  # Assuming the company with ID 1 exists or is created prior to contract creation
    "total_value": 10000,
    "date_awarded": "2023-10-01"
}

# He